In [16]:
import pandas as pd
from sklearn.calibration import LabelEncoder

# 1. Load the data
df = pd.read_csv('./megablunders.csv')

In [17]:
# use lable encoder to encode the error
label_encoder = LabelEncoder()
df['error'] = label_encoder.fit_transform(df['error'])

In [18]:
#  vectorize the text 
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [19]:
df["original_sentence"] = model.encode(df["original_sentence"])

In [20]:
df.head()

,original_sentence,error
0,-0.078433,3
1,0.004779,4
2,-0.069395,8
3,-0.021092,0
4,-0.030555,7


In [21]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['original_sentence'], df['error'], test_size=0.1, random_state=42)

In [30]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [34]:
# Modify the model to work with 2D input
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[0],)))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(9, activation='softmax'))

In [35]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
history = model.fit(np.expand_dims(X_train, axis=0),y_train, batch_size=48, epochs=20, validation_data=(X_test, y_test))

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1
'y' sizes: 107
